In [9]:
import numpy as np
import numpy.random as rn
import matplotlib.pyplot as plt
import seaborn as sns

import scipy.stats as st

import sys
from path import Path
sys.path.append('..')
from apf.models.bpmf import BPMF

In [43]:
seed = 222
rn.seed(seed)

n_samps = 100
n_feats = 200
n_comps = 10

eps = 0.1
b = 10

b = rn.gamma(eps, 1./eps)
Theta_IK = rn.gamma(eps, 1./(eps * b), size=(n_samps, n_comps))
Phi_KJ = rn.dirichlet(np.ones(n_feats), size=n_comps)
Lambda_K = rn.gamma(eps, 1./eps, size=n_comps)
Mu_IJ = Theta_IK.dot(Phi_KJ)

Y_IJ = rn.poisson(Mu_IJ)                        # count data
B_IJ = (Y_IJ > 0).astype(int)                   # binarized data
mask_IJ = rn.binomial(1, 0.1, size=Y_IJ.shape)  # randomly make 10% missing

count_data = np.ma.MaskedArray(Y_IJ, mask_IJ.astype(bool))  # create masked count data
binary_data = np.ma.MaskedArray(B_IJ, mask_IJ.astype(bool)) # create masked binary data

In [23]:
K = 15
model = BPMF(n_samps=count_data.shape[0],
             n_feats=count_data.shape[1],
             n_comps=K,
             binary=False,
             n_threads=3)

burnin = 1000  # 1000 iterations of burnin
model.fit(count_data, 
          n_itns=burnin, 
          initialize=True,
          verbose=250)

n_epochs = 100  # number of Gibbs sampling epochs
n_itns = 50     # number of Gibbs iterations per epoch

prob_IJ = np.zeros(count_data.shape)
for epoch in range(n_epochs):
    model.fit(count_data, 
              n_itns=n_itns, 
              initialize=False,
              verbose=50)
    
    pred_IJ = model.reconstruct()
    prob_IJ += st.poisson.pmf(count_data.data, pred_IJ)
prob_IJ /= float(n_epochs)

mask = count_data.mask
test_data = count_data.data[mask]
train_data = count_data.data[~mask]

test_ppd = np.exp(np.mean(np.log(prob_IJ[mask])))
train_ppd = np.exp(np.mean(np.log(prob_IJ[~mask])))

print(f'Pointwise predictive density on training data: {train_ppd}')
print(f'Pointwise predictive density on test data: {test_ppd}')


INITIALIZING...


STARTING INFERENCE...

0.0360ms: sampling b
1.4708ms: sampling Lambda_K
0.9370ms: sampling Theta_IK
0.3889ms: sampling Phi_KJ
7.6892ms: sampling Y_PK
ITERATION 250

0.0219ms: sampling b
1.2150ms: sampling Lambda_K
0.7570ms: sampling Theta_IK
0.3231ms: sampling Phi_KJ
7.5612ms: sampling Y_PK
ITERATION 500

0.0210ms: sampling b
1.2081ms: sampling Lambda_K
1.3211ms: sampling Theta_IK
0.7460ms: sampling Phi_KJ
7.9103ms: sampling Y_PK
ITERATION 750

0.0331ms: sampling b
1.1110ms: sampling Lambda_K
1.0180ms: sampling Theta_IK
0.3541ms: sampling Phi_KJ
7.7429ms: sampling Y_PK
ITERATION 1000


STARTING INFERENCE...

0.0219ms: sampling b
1.1382ms: sampling Lambda_K
0.9570ms: sampling Theta_IK
0.3531ms: sampling Phi_KJ
7.6058ms: sampling Y_PK
ITERATION 1050


STARTING INFERENCE...

0.0219ms: sampling b
1.3189ms: sampling Lambda_K
0.9809ms: sampling Theta_IK
0.3459ms: sampling Phi_KJ
7.7100ms: sampling Y_PK
ITERATION 1100


STARTING INFERENCE...

0.0222ms: sampling b
1.1051ms: 

0.0219ms: sampling b
1.2000ms: sampling Lambda_K
0.7958ms: sampling Theta_IK
0.3443ms: sampling Phi_KJ
7.6020ms: sampling Y_PK
ITERATION 3350


STARTING INFERENCE...

0.0210ms: sampling b
1.0581ms: sampling Lambda_K
0.9310ms: sampling Theta_IK
0.4210ms: sampling Phi_KJ
7.6339ms: sampling Y_PK
ITERATION 3400


STARTING INFERENCE...

0.0231ms: sampling b
1.1053ms: sampling Lambda_K
1.0638ms: sampling Theta_IK
0.4239ms: sampling Phi_KJ
7.6947ms: sampling Y_PK
ITERATION 3450


STARTING INFERENCE...

0.0229ms: sampling b
1.0002ms: sampling Lambda_K
1.0688ms: sampling Theta_IK
0.4029ms: sampling Phi_KJ
7.6041ms: sampling Y_PK
ITERATION 3500


STARTING INFERENCE...

0.0219ms: sampling b
1.0760ms: sampling Lambda_K
0.8910ms: sampling Theta_IK
0.4542ms: sampling Phi_KJ
7.6501ms: sampling Y_PK
ITERATION 3550


STARTING INFERENCE...

0.0241ms: sampling b
1.0340ms: sampling Lambda_K
0.9360ms: sampling Theta_IK
0.3562ms: sampling Phi_KJ
7.7150ms: sampling Y_PK
ITERATION 3600


STARTING INFERENCE...

0.0231ms: sampling b
0.9880ms: sampling Lambda_K
0.9429ms: sampling Theta_IK
0.3371ms: sampling Phi_KJ
7.7252ms: sampling Y_PK
ITERATION 5850


STARTING INFERENCE...

0.0219ms: sampling b
1.1010ms: sampling Lambda_K
0.8769ms: sampling Theta_IK
0.3340ms: sampling Phi_KJ
7.5929ms: sampling Y_PK
ITERATION 5900


STARTING INFERENCE...

0.0200ms: sampling b
1.3130ms: sampling Lambda_K
0.9258ms: sampling Theta_IK
0.3738ms: sampling Phi_KJ
7.5891ms: sampling Y_PK
ITERATION 5950


STARTING INFERENCE...

0.0250ms: sampling b
1.0791ms: sampling Lambda_K
0.9928ms: sampling Theta_IK
0.3538ms: sampling Phi_KJ
7.5567ms: sampling Y_PK
ITERATION 6000

Pointwise predictive density on training data: 0.27395914321085585
Pointwise predictive density on test data: 0.22617194965412216


In [44]:
K = 15
model = BPMF(n_samps=binary_data.shape[0],
             n_feats=binary_data.shape[1],
             n_comps=K,
             binary=True,
             n_threads=3)

burnin = 1000  # 1000 iterations of burnin
model.fit(binary_data, 
          n_itns=burnin, 
          initialize=True,
          verbose=250)

n_epochs = 100  # number of Gibbs sampling epochs
n_itns = 50     # number of Gibbs iterations per epoch

prob_IJ = np.zeros(binary_data.shape)
for epoch in range(n_epochs):
    model.fit(binary_data, 
              n_itns=n_itns, 
              initialize=False,
              verbose=50)
    
    pred_IJ = -np.expm1(-model.reconstruct())
    prob_IJ += st.bernoulli.pmf(binary_data.data, pred_IJ)
prob_IJ /= float(n_epochs)

mask = binary_data.mask
test_data = binary_data.data[mask]
train_data = binary_data.data[~mask]

test_ppd = np.exp(np.mean(np.log(prob_IJ[mask])))
train_ppd = np.exp(np.mean(np.log(prob_IJ[~mask])))

print(f'Pointwise predictive density on training data: {train_ppd}')
print(f'Pointwise predictive density on test data: {test_ppd}')


INITIALIZING...


STARTING INFERENCE...

0.0210ms: sampling b
1.1311ms: sampling Lambda_K
0.9921ms: sampling Theta_IK
0.3550ms: sampling Phi_KJ
7.9479ms: sampling Y_PK
ITERATION 250

0.0219ms: sampling b
1.1320ms: sampling Lambda_K
0.8929ms: sampling Theta_IK
0.3181ms: sampling Phi_KJ
7.9591ms: sampling Y_PK
ITERATION 500

0.0219ms: sampling b
1.0402ms: sampling Lambda_K
1.0989ms: sampling Theta_IK
0.3841ms: sampling Phi_KJ
8.1189ms: sampling Y_PK
ITERATION 750

0.0200ms: sampling b
1.5218ms: sampling Lambda_K
1.0860ms: sampling Theta_IK
0.3860ms: sampling Phi_KJ
8.1708ms: sampling Y_PK
ITERATION 1000


STARTING INFERENCE...

0.0229ms: sampling b
1.1261ms: sampling Lambda_K
0.9890ms: sampling Theta_IK
0.3459ms: sampling Phi_KJ
8.1389ms: sampling Y_PK
ITERATION 1050


STARTING INFERENCE...

0.0241ms: sampling b
1.0240ms: sampling Lambda_K
0.9868ms: sampling Theta_IK
0.3409ms: sampling Phi_KJ
8.2850ms: sampling Y_PK
ITERATION 1100


STARTING INFERENCE...

0.0203ms: sampling b
1.2560ms: 

0.0231ms: sampling b
1.0290ms: sampling Lambda_K
0.9890ms: sampling Theta_IK
0.3459ms: sampling Phi_KJ
8.1041ms: sampling Y_PK
ITERATION 3350


STARTING INFERENCE...

0.0222ms: sampling b
0.9930ms: sampling Lambda_K
1.0638ms: sampling Theta_IK
0.3221ms: sampling Phi_KJ
8.2021ms: sampling Y_PK
ITERATION 3400


STARTING INFERENCE...

0.0229ms: sampling b
1.1480ms: sampling Lambda_K
0.9439ms: sampling Theta_IK
0.4771ms: sampling Phi_KJ
8.0833ms: sampling Y_PK
ITERATION 3450


STARTING INFERENCE...

0.0222ms: sampling b
1.1053ms: sampling Lambda_K
0.8688ms: sampling Theta_IK
0.3338ms: sampling Phi_KJ
8.2009ms: sampling Y_PK
ITERATION 3500


STARTING INFERENCE...

0.0422ms: sampling b
1.0679ms: sampling Lambda_K
1.0421ms: sampling Theta_IK
0.3169ms: sampling Phi_KJ
8.0731ms: sampling Y_PK
ITERATION 3550


STARTING INFERENCE...

0.0210ms: sampling b
1.1399ms: sampling Lambda_K
0.9718ms: sampling Theta_IK
0.3111ms: sampling Phi_KJ
8.0750ms: sampling Y_PK
ITERATION 3600


STARTING INFERENCE...

0.0231ms: sampling b
1.0707ms: sampling Lambda_K
0.8550ms: sampling Theta_IK
0.4559ms: sampling Phi_KJ
8.4779ms: sampling Y_PK
ITERATION 5850


STARTING INFERENCE...

0.0219ms: sampling b
1.1110ms: sampling Lambda_K
0.9611ms: sampling Theta_IK
0.4327ms: sampling Phi_KJ
8.3930ms: sampling Y_PK
ITERATION 5900


STARTING INFERENCE...

0.0219ms: sampling b
1.2038ms: sampling Lambda_K
1.0052ms: sampling Theta_IK
0.2983ms: sampling Phi_KJ
8.2021ms: sampling Y_PK
ITERATION 5950


STARTING INFERENCE...

0.0210ms: sampling b
1.2729ms: sampling Lambda_K
0.9310ms: sampling Theta_IK
0.3181ms: sampling Phi_KJ
8.1711ms: sampling Y_PK
ITERATION 6000

Pointwise predictive density on training data: 0.7724739613414147
Pointwise predictive density on test data: 0.6240841245979063
